In [142]:
import requests
from bs4 import BeautifulSoup
import lxml
import re

In [147]:
barcode = input('barcode를 입력하세요:')

def get_reple(pageNumber=1):
    
    url = f'http://www.kyobobook.co.kr/product/productSimpleReviewSort.laf?gb=klover&ejkGb=KOR&mallGb=&sortType=like&orderType=order&barcode={barcode}&pageNumber={pageNumber}'

    resp = requests.get(url)
    soup = BeautifulSoup(resp.text,"lxml")
    page=soup.select_one('ul.board_list').select('div.comment_wrap')
    
    # r=> 댓글, s> 평점
    r, s =[],[]
    
    for dl in page:
        reviews=dl.select('div.txt')  
        for review in reviews:
            #print('댓글:', review.text)
            review=review.text
            review = review.replace('\n','')
            review = review.replace('\r','')
            if int(dl.span.text) >=4:
                r.append(review)
                s.append(1) # 긍정
            elif int(dl.span.text) <=2:
                r.append(review)
                s.append(0) #부정

    return r,s

barcode를 입력하세요:9791187962335 


In [148]:
get_reple(1)

(['',
  '삶에 도움이 되는 조언들이 가득해서 좋았어요 !',
  '뭔가 꽉 막힌 아버지가 하는 말같아요... 잘 안읽히네요 나중에 다시 읽어봐야겠어요',
  '이 분류의 다른책을 봤었는데확실히 이책에 뭔가 더 얻음이 많고공감됨이 많아요',
  ''],
 [1, 1, 0, 1, 1])

In [138]:
#전체페이지 개수 알아내기

barcode = input('barcode를 입력하세요:')

url = f'http://www.kyobobook.co.kr/product/productSimpleReviewSort.laf?gb=klover&ejkGb=KOR&mallGb=&sortType=like&orderType=order&barcode={barcode}&pageNumber=1'
resp = requests.get(url)
soup = BeautifulSoup(resp.text,"lxml")
last_page=[]

pages =soup.find('div',{'class':'paging_num'}).findAll('a',{'class':'pad'})
#pages
for i in pages:
    if 'href' in i.attrs:
        pages=i.attrs['href']
        last_page.append(pages)
last_page_num=last_page[1]
last_page_num=re.sub('[^0-9]','',last_page_num).strip()
last_page_num=int(last_page_num)
#type(last_page_num)
last_page_num


barcode를 입력하세요:9791187962335 


20

In [139]:
import time
reple, score = [], []

#여기서 i는 페이지 수
for i in range (1,last_page_num):
    time.sleep(1)
    print('요청횟수:',i, end='\r')
    get_reple(i)
    r, s = get_reple(i)
    reple += r    
    score += s

In [141]:
#데이터 개수 확인
len(reple),len(score)

(82, 82)

In [86]:
#데이터확인
print(reple,score)

['고등학교를 졸업하는 우리 아이에게 선물하고 싶은 책.  특히 사회초년생들에게 권하고 싶어요', '“돈 생기면 그때 생각하자.”라는 생각이 당신의 ‘머니플랜’을 늦춘다라는 말이 인상적이네요.', '여러모로 많은 도움을 줄 수 있을 것 같아 만나보고 싶어요! 부자가 되기 위해 열심히 노력 중입니다. 감사합니다.', '부자가 될 수 있을 것 같은 기대감이 들어 구매합니다. ^^', '책의 소제목만 봐도 두근두근하네요. 2020년엔 한걸음더 나아가고 싶어요!!!', '이 책을 통해 하버드 출신 부자들의 마인드를 배우고 싶습니다.', '하버드 출신 부자들의 삶의 태도와 정신들이 무엇인지 궁금하네요.', '책 참 좋네요.', '돈관리가 서툰데 기대됩니다^^', '멋지네요^^', '기대되는 책 :)', '하버드 출신들은 어떻게 부자로 살게 되는지 너무 궁금하네요.', '읽고 또 읽으며 생각을 곱씹어야 할 책입니다', '돈관리는 장기적인 시점에서 생각을 할줄 알아야 하는데, 하루벌어 먹고 살기 힘드니. 저도 자산관리하는 날이 오겠죠', '돈을 모으고 돈을 잘 쓰는 것 이것이 자산관리의 기본입니다', '확실히 열심히 살도록 자극이 되는 책이다.', '자산관리 상식을 넓혀주는 책이라 좋아요', '자산관리.. 지금이라도 읽어서 다행이네요.', '재미나게 읽고 있습니다. 스토리구성이 좋네요^^', 'good^^잘 읽고 있습니다. 소중한 사람에게 추천하고 읽히고 싶은 책입니다', '부자가 될 수 있게 해주는 영원한 고전 같아요^^', '너무 좋네요. 쭉쭉 읽고 있어요', '이 책을 인생 올해의 책으로 정했다. 이 책을 만난 건 나에게 행운!!', '이 책을 읽을 수 있음에 감사함을 느꼈습니다... 정말 좋아요~~~', '하버드 부자수업으로 올해는 나도 자산관리에 힘을 써볼렵니다', '', '후배에게 선물하려고 한권 더 샀어요~~~', '정말 유용한 책인 것 같아요 다른 분들께도 꼭 추천드려요', '완죤 좋아요~ 강츄해요~!!!', '친구에게 선물하려고 오늘 한권 더 샀어요.^^'

In [ ]:
#평점이 1이면 긍정, 0이면 부정, 데이터 저장
import pandas as pd
df=pd.DataFrame([reple,score]).T
df=df.rename({0:'댓글',1:'평점'}, axis='columns')
df

In [ ]:
#데이터 csv로 저장
df.to_csv('아들아, 삶에 지치고 힘들 때 이 글을 읽어라.csv')